In [16]:
import telebot
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import re

In [26]:
# Загрузка модели и токенизатора
model_name = "cointegrated/rubert-tiny-sentiment-balanced"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [27]:
# Устройство (CPU или GPU)
device = 0 if torch.cuda.is_available() else -1

# Создание пайплайна для анализа тональности
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)

# Список плохих слов 
bad_words = ["дура", "дурак"]  

In [28]:
# Предобработка текста
def preprocess_text(text: str) -> str:
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление знаков препинания
    return text.strip()


In [29]:
# Функция для анализа тональности
def analyze_sentiment(text: str) -> dict:
    preprocessed_text = preprocess_text(text)
    result = sentiment_analysis(preprocessed_text)[0]
    return {"label": result['label'], "score": result['score']}

In [30]:
# Функция для поиска плохих слов
def find_bad_words(text: str) -> list:
    text = preprocess_text(text)
    words = text.split()
    return [word for word in words if word in bad_words]

In [31]:
# Форматирование результата
def format_response(message_text: str, sentiment: dict, bad_words: list) -> str:
    response = f"Сообщение: {message_text}\n"
    response += f"Тональность: {sentiment['label']} (оценка: {sentiment['score']:.2f})\n"
    if bad_words:
        response += f"Плохие слова: {', '.join(bad_words)}\n"
    else:
        response += "Плохих слов не найдено\n"
    return response

In [32]:
# токен Telegram бота
TELEGRAM_TOKEN = ' '  # здесь нужно вписать свой telegram-токен
bot = telebot.TeleBot(TELEGRAM_TOKEN)

In [33]:
# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, "Бот запущен. Жду сообщений")

In [34]:
# Обработчик текстовых сообщений
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    sentiment = analyze_sentiment(message.text)
    bad_words_found = find_bad_words(message.text)
    
    # Форматируем ответ
    formatted_response = format_response(message.text, sentiment, bad_words_found)
    
    bot.send_message(message.chat.id, formatted_response)

In [35]:
# Запуск бота
bot.polling(none_stop=True)

D:\ProgramData\anaconda3\envs\myenv\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
